In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.loc[(train['marital.status'] == 'Widowed') & (train['relationship'] == 'Unmarried'), 'relationship'] = 'Widowed'
test.loc[(test['marital.status'] == 'Widowed') & (test['relationship'] == 'Unmarried'), 'relationship'] = 'Widowed'


In [30]:
y_train = train['income']
y_test = test['income']

In [31]:
# train['occupation'].unique()
# occupation_dict = {
#     "Exec-managerial": "Executive, administrative, and managerial",
#     "Other-service": "Service occupations",
#     "Machine-op-inspct": "Operators, fabricators, and laborers",
#     "Transport-moving": "Precision production, craft, and repair",
#     "Prof-specialty": "Professional specialty",
#     "Craft-repair": "Precision production, craft, and repair",
#     "Adm-clerical": "Administrative support, including clerical",
#     "Handlers-cleaners": "Service occupations",
#     "Sales": "Sales occupations",
#     "Farming-fishing": "Farming, forestry, and fishing",
#     "Protective-serv": "Service occupations",
#     "Tech-support": "Technicians and related support",
#     "Priv-house-serv": "Service occupations",
#     "Armed-Forces": "Service occupations",
#     np.nan : "Part-timer"
# }
# job_values_dict = {
#     "Executive, administrative, and managerial": 76,
#     "Professional specialty": 73.5,
#     "Technicians and related support": 57.8,
#     "Sales occupations": 38.7,
#     "Administrative support, including clerical": 40,
#     "Service occupations": 27.3,
#     "Precision production, craft, and repair": 58.2,
#     "Operators, fabricators, and laborers": 39.6,
#     "Farming, forestry, and fishing": 30.1,
#     "Part-timer" : 30
# }
# # 새로운 열 생성
# train['wage_avg'] = train['occupation'].replace(occupation_dict)
# train['wage_avg'] = train['wage_avg'].replace(job_values_dict)

# test['wage_avg'] = test['occupation'].replace(occupation_dict)
# test['wage_avg'] = test['wage_avg'].replace(job_values_dict)
# # # occupation 열의 값 변경
# # train['occupation'] = train['occupation'].replace(occupation_dict)
# # train['occupation'] = train['occupation'].replace(job_values_dict)

# # test['occupation'] = test['occupation'].replace(occupation_dict)
# # test['occupation'] = test['occupation'].replace(job_values_dict)

In [32]:
def ratio_sort(df, column):
    # 각 범주에 대해 'income'이 1인 비율 계산
    category_stats = df.groupby(column)['income'].agg(['sum', 'count'])  # 'sum'은 income이 1인 개수, 'count'는 전체 개수
    category_stats['ratio'] = category_stats['sum'] / category_stats['count']  # 비율 계산
    
    # 비율에 따라 내림차순 정렬
    sorted_category_stats = category_stats.sort_values(by='ratio', ascending=False)
    
    
    return sorted_category_stats[['ratio']]

def show_gap(column):
    print(ratio_sort(test, column),"\n", ratio_sort(train, column))
    gap = ratio_sort(test, column)- ratio_sort(train, column)
    gaps = gap.sort_values(by='ratio',ascending = False)
    print(gaps)

def replace_country(df):
    # native.country 열에서 'United-States'가 아닌 값은 'others'로 변경
    df['native.country'] = df['native.country'].apply(lambda x: 'United-States' if x == 'United-States' else 'others')
    return df
def replace_relationship(df):
    # native.country 열에서 'United-States'가 아닌 값은 'others'로 변경
    df['relationship'] = df['relationship'].apply(lambda x: 'married' if x == 'Husband'or x =='Wife' else 'others')
    return df
replace_country(train)
replace_country(test)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,45,Federal-gov,207107,Assoc-acdm,12,Divorced,Adm-clerical,Unmarried,Asian-Pac-Islander,Male,0,2080,40,others,0
1,22,Self-emp-not-inc,197387,HS-grad,9,Never-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States,0
2,39,Private,111268,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,0
3,35,Private,98283,Prof-school,15,Never-married,Prof-specialty,Not-in-family,Asian-Pac-Islander,Male,0,0,40,others,1
4,40,Private,224232,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7536,20,Private,368852,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,20,United-States,0
7537,52,Private,164135,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,1
7538,50,Private,226497,Some-college,10,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,45,United-States,1
7539,26,Private,290213,Some-college,10,Separated,Farming-fishing,Own-child,White,Male,0,0,40,United-States,0


In [33]:
 def get_category(ratio):
        if ratio > 0.7: return 'very_high'
        elif 0.5 < ratio <= 0.7: return 'high'
        elif 0.3 < ratio <= 0.5: return 'middle_high'
        elif 0.25 < ratio <= 0.3: return 'middle'
        elif 0.2 < ratio <= 0.25: return 'middle_low'
        elif 0.1 < ratio <= 0.2: return 'low'
        else: return 'very_low'
            
def categorize_column(df, column):

    if df.shape[0] >= 10000:
        print(train.shape)
        ratio_ = ratio_sort(train, column)
        ratio_dict = ratio_['ratio'].to_dict()
        print('train')
    else:
        ratio_ = ratio_sort(test, column)
        ratio_dict = ratio_['ratio'].to_dict() 
        print('test')

    category_mapping = {key: get_category(ratio) for key, ratio in ratio_dict.items()}
    
    df[column+'_categorize'] = df[column].map(category_mapping)

    return df

In [34]:
test

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,45,Federal-gov,207107,Assoc-acdm,12,Divorced,Adm-clerical,Unmarried,Asian-Pac-Islander,Male,0,2080,40,others,0
1,22,Self-emp-not-inc,197387,HS-grad,9,Never-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States,0
2,39,Private,111268,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,0
3,35,Private,98283,Prof-school,15,Never-married,Prof-specialty,Not-in-family,Asian-Pac-Islander,Male,0,0,40,others,1
4,40,Private,224232,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7536,20,Private,368852,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,20,United-States,0
7537,52,Private,164135,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,1
7538,50,Private,226497,Some-college,10,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,45,United-States,1
7539,26,Private,290213,Some-college,10,Separated,Farming-fishing,Own-child,White,Male,0,0,40,United-States,0


In [35]:
train.fillna("NULL", inplace = True)
test.fillna("NULL", inplace = True)

In [36]:
train_categorize = train.select_dtypes(include=["object"])
train_numeric = train.select_dtypes(include=["number"])
train_numeric.drop(columns = ['fnlwgt','income'], inplace = True)

test_categorize = test.select_dtypes(include=["object"])
test_numeric = test.select_dtypes(include=["number"])
test_numeric.drop(columns = ['fnlwgt', 'income'], inplace = True)

In [37]:
target = []
for i in target:
    categorize_column(train_categorize,i)
    categorize_column(test_categorize,i)
train_categorize

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
0,Private,Bachelors,Divorced,Exec-managerial,Unmarried,White,Female,United-States
1,Private,HS-grad,Never-married,Other-service,Own-child,White,Female,others
2,Private,HS-grad,Widowed,Other-service,Not-in-family,White,Female,United-States
3,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
4,Private,11th,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
...,...,...,...,...,...,...,...,...
25015,NULL,Bachelors,Married-civ-spouse,NULL,Wife,White,Female,United-States
25016,NULL,Bachelors,Never-married,NULL,Not-in-family,Asian-Pac-Islander,Female,United-States
25017,NULL,Doctorate,Married-civ-spouse,NULL,Husband,White,Male,United-States
25018,NULL,HS-grad,Separated,NULL,Not-in-family,Black,Female,United-States


In [38]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(train_categorize)

train_encoded = encoder.transform(train_categorize)
test_encoded = encoder.transform(test_categorize)

# 결과를 DataFrame으로 변환하여 열 이름 지정
train_categorize = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(train_categorize.columns))
test_categorize = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(test_categorize.columns))

In [39]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# train_numeric = pd.DataFrame(scaler.fit_transform(train_numeric), columns = train_numeric.columns)

# test_numeric = pd.DataFrame(scaler.transform(test_numeric), columns = test_numeric.columns)

In [40]:
train = pd.concat([train_numeric, train_categorize], axis=1)
test = pd.concat([test_numeric, test_categorize], axis=1)

In [42]:
try:
    y_train = train['income']
    X_train = train.drop(columns = 'income')
    y_test = test['income']
    X_test = test.drop(columns = 'income')
except:
    X_train = train
    X_test = test

In [43]:
train

,age,education.num,capital.gain,capital.loss,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_NULL,workclass_Never-worked,workclass_Private,...,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male,native.country_United-States,native.country_others
0,41,13,0,0,45,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,20,9,0,0,40,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,73,9,0,0,37,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,56,10,7688,0,50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,53,7,0,0,40,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25015,35,13,0,0,55,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
25016,30,13,0,0,99,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
25017,71,16,0,0,10,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
25018,41,9,0,0,32,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [46]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=2000, learning_rate=0.005)
lgbm.fit(X_train, y_train)

# 예측
pred = lgbm.predict(X_test)

# accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred)
print(accuracy)

[LightGBM] [Info] Number of positive: 5943, number of negative: 19077
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002856 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 449
[LightGBM] [Info] Number of data points in the train set: 25020, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.237530 -> initscore=-1.166269
[LightGBM] [Info] Start training from score -1.166269
0.8712372364407903


In [19]:
"""
0.8712372364407903
"""

'\n0.8712372364407903\n'